In [1]:
!conda install -c conda-forge gdcm -y

Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - gdcm


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.8.5                |   py37hc8dfbb8_1         3.0 MB  conda-forge
    gdcm-2.8.9                 |   py37h71b2a6d_0         3.4 MB  conda-forge
    libjpeg-turbo-2.0.3        |       h516909a_1        1022 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         7.5 MB

The following NEW packages will be INSTALLED:

  gdcm               conda-forge/li

In [2]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm 
import gc
import glob, os
import pydicom
from PIL import Image
import matplotlib.pyplot as plt
import gdcm
import pickle
import skimage.measure
import cv2

In [3]:
train = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/train.csv')
test = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/test.csv')

train_out = pd.DataFrame()
img_array = []
instance_array = []
img_dict = {}
for Patient in tqdm(train.Patient.unique()) :
    for filename in glob.glob('../input/osic-pulmonary-fibrosis-progression/train/'+Patient+'/*'):
#         img = skimage.measure.block_reduce(pydicom.dcmread(filename).pixel_array, (3,3), np.max)
        d = pydicom.dcmread(filename)
        instance = d.InstanceNumber
        img = cv2.resize((d.pixel_array * d.RescaleSlope + d.RescaleIntercept)/1000, (512, 512))
#         img = img.flatten() #To avoid imagepooling
        img = skimage.measure.block_reduce(img, (2,2), np.max).flatten()
        img_array.append(img)
        instance_array.append(instance)
    df_tmp = pd.DataFrame(img_array).astype('float16')
    df_tmp['Instance'] = instance_array
    df_tmp['Patient'] = Patient
    train_out = pd.concat([train_out, df_tmp])
#     img_dict[Patient] = img_array
    img_array = []
    instance_array = []
gc.collect()

for Patient in tqdm(test.Patient.unique()) :
    for filename in glob.glob('../input/osic-pulmonary-fibrosis-progression/test/'+Patient+'/*'):
#         img = skimage.measure.block_reduce(pydicom.dcmread(filename).pixel_array, (3,3), np.max)
        d = pydicom.dcmread(filename)
        instance = d.InstanceNumber
        img = cv2.resize((d.pixel_array * d.RescaleSlope + d.RescaleIntercept)/1000, (512, 512))
#         img = img.flatten() #To avoid imagepooling
        img = skimage.measure.block_reduce(img, (2,2), np.max).flatten()
        img_array.append(img)
        instance_array.append(instance)
    df_tmp = pd.DataFrame(img_array).astype('float16')
    df_tmp['Instance'] = instance_array
    df_tmp['Patient'] = Patient
    train_out = pd.concat([train_out, df_tmp])
#     img_dict[Patient] = img_array
    img_array = []
    instance_array = []
gc.collect()
        
print('Train Done')


Train Done


In [4]:
train_out.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34287 entries, 0 to 407
Columns: 65538 entries, 0 to Patient
dtypes: float16(65536), int64(1), object(1)
memory usage: 4.2+ GB


In [5]:
train_out.head()

,0,1,2,3,4,5,6,7,8,9,...,65528,65529,65530,65531,65532,65533,65534,65535,Instance,Patient
0,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,...,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,3,ID00007637202177411956430
1,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,...,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,28,ID00007637202177411956430
2,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,...,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,20,ID00007637202177411956430
3,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,...,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,13,ID00007637202177411956430
4,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,...,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,-3.023438,15,ID00007637202177411956430


In [6]:
train_out.to_pickle("train_out.pkl")

In [7]:
test_out = pd.DataFrame()
i=0
img_array = []
instance_array = []
img_dict = {}
for Patient in tqdm(test.Patient.unique()) :
    for filename in glob.glob('../input/osic-pulmonary-fibrosis-progression/test/'+Patient+'/*.dcm'):
#         img = skimage.measure.block_reduce(pydicom.dcmread(filename).pixel_array, (3,3), np.max)
        d = pydicom.dcmread(filename)
        instance = d.InstanceNumber
#         img = cv2.resize((d.pixel_array - d.RescaleIntercept) / (d.RescaleSlope * 1000), (512, 512))
        img = cv2.resize((d.pixel_array * d.RescaleSlope + d.RescaleIntercept)/1000, (512, 512))
#         img = img.flatten() #To avoid imagepooling
        img = skimage.measure.block_reduce(img, (2,2), np.max).flatten()
        img_array.append(img)
        instance_array.append(instance)
    df_tmp = pd.DataFrame(img_array).astype('float16')
    df_tmp['Instance'] = instance_array
    df_tmp['Patient'] = Patient
    test_out = pd.concat([test_out, df_tmp])
#     img_dict[Patient] = img_array
    img_array = []
    instance_array = []
    gc.collect()

print('test Done')


test Done


In [8]:
test_out.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1261 entries, 0 to 407
Columns: 65538 entries, 0 to Patient
dtypes: float16(65536), int64(1), object(1)
memory usage: 157.7+ MB


In [9]:
test_out

,0,1,2,3,4,5,6,7,8,9,...,65528,65529,65530,65531,65532,65533,65534,65535,Instance,Patient
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3,ID00419637202311204720264
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,28,ID00419637202311204720264
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,20,ID00419637202311204720264
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13,ID00419637202311204720264
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,15,ID00419637202311204720264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403,-1.024414,-1.024414,-1.024414,-1.024414,-1.024414,-1.024414,-1.024414,-1.024414,-1.024414,-1.024414,...,-0.984863,-1.024414,-1.024414,-1.024414,-1.024414,-1.024414,-1.024414,-1.024414,110,ID00426637202313170790466
404,-1.024414,-1.024414,-1.024414,-1.024414,-1.024414,-1.024414,-1.024414,-1.024414,-1.024414,-1.024414,...,-0.983887,-1.024414,-1.024414,-1.024414,-1.024414,-1.024414,-1.024414,-1.024414,305,ID00426637202313170790466
405,-1.024414,-1.024414,-1.024414,-1.024414,-1.024414,-1.024414,-1.024414,-1.024414,-1.024414,-1.024414,...,-1.024414,-1.024414,-1.024414,-1.024414,-1.024414,-1.024414,-1.024414,-1.024414,107,ID00426637202313170790466
406,-1.024414,-1.024414,-1.024414,-1.024414,-1.024414,-1.024414,-1.024414,-1.024414,-1.024414,-1.024414,...,-0.958008,-1.024414,-1.024414,-1.024414,-1.024414,-1.024414,-1.024414,-1.024414,135,ID00426637202313170790466


In [10]:
test_out.to_pickle("test_out.pkl")

In [11]:

# \pd.concat([train_out, test_out], ignore_index=True).to_pickle("train_out.pkl")